In [1]:
from src.utils import *
from sklearn import svm

In [2]:
X, Y = load(), load(X=False)

In [3]:
def kernel(X, Z):
    return (X[:, None, :] == Z[None, :, :]).sum(-1)

In [4]:
clf = svm.SVC(kernel=kernel)
evaluate(clf, X, Y)

(0.5645, 0.01691153452528773)